In [4]:
from pylab import *
import numpy as np
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(10,5))
rc('font',size=20)
rc('text',usetex=True)

In [5]:
z = np.logspace(-1, 4, 1000)

In [6]:
data = np.genfromtxt('SCPUnion2.1_mu_vs_z.txt')
zs = data.T[1]
mm = data.T[2]
dm = data.T[3]
dlabel = 'SCP_2.1'

In [7]:
dpc = 10.**(mm/5.+1.)
dMpc = dpc / 10.**6
dMe = 10.**((mm+dm)/5.+1.-6.) - dMpc
c = 3.e5
indx = np.where(zs < 0.05)
H0 = (c/dMpc[indx]*zs[indx]).mean()
sH0 = 'H0 = '+str(round(H0))+' km/s/Mpc'

dz = 0.001
zz = np.arange(dz,2.0,dz)
ds1 = c/H0*zz
ds2 = c/H0*zz*np.sqrt(1.+zz)
OmM = 0.3 
OmL = 1. - OmM 
ds3 = c/H0*np.cumsum(1./np.sqrt(OmM*(1+zz)**3+OmL))*dz*(1.+zz)

m1 = (np.log10(ds1)+5.)*5. 
m2 = (np.log10(ds2)+5.)*5. 
m3 = (np.log10(ds3)+5.)*5. 


dp2 = np.interp(zs,zz,ds2)
dp3 = np.interp(zs,zz,ds3)

chisq2 = np.sum( ((dMpc-dp2)/dMe)**2 ) / dMpc.size
chisq3 = np.sum( ((dMpc-dp3)/dMe)**2 ) / dMpc.size
print 'chisq2, chisq3 = ',chisq2,chisq3


chisq2, chisq3 =  0.9805546495975834 0.9620136723761287


In [8]:
plt.figure()
plt.xscale('log')
plt.xlim(1e-2,2)
plt.ylim(33,46)
plt.xlabel('Redshift')
plt.ylabel('Distance Modulus')
plt.show()

OSError: [Errno 2] No such file or directory: 'latex'

<Figure size 432x288 with 1 Axes>

In [9]:
class Data():
    def __init__(self, xvals=None, yvals=None, errors = None, model=None, prior=False):
        self.prior = prior
        self.model = model
        self.xvals = xvals
        self.yvals = yvals
        if not self.prior:
            if np.size(np.shape(errors)) == 1:
                self.covar=np.zeros((np.size(errors),np.size(errors)))
                self.covar[np.arange(np.size(errors)),np.arange(np.size(errors))]=errors**2
            else:
                self.covar = errors
            self.invcov = np.linalg.inv(self.covar)
    
    def __call__(self,*pars):
        if  not self.prior:
            val=self.model(self.xvals,pars[0])
            chi2=np.dot(np.dot(self.yvals-val,self.invcov),self.yvals-val)
        else:
            chi2 = self.model(self.xvals, pars[0])
        return(-0.5*chi2)
                

In [10]:
def musn1a(z, cosmo):
    dlum = lumdist(z, cosmo)*1e6
    return(5*np.log10(dlum)-5+5*np.log10(cosmo['h']/0.7))

In [12]:
def lumdist(z,cosmo,zres=0.001,accurate=False):
    return(propdist(z,cosmo,zres=zres,accurate=accurate)*(1+z))


In [14]:
def propdist(z,cosmo,zres=0.001,accurate=False):
    ### z range for integration
    zmax=np.max(z)
    if zmax < zres:
        nb=101
    else:
        nb=zmax/zres+1
    zvals=np.linspace(0.,zmax,nb)
    ### integrate
    cumulative=np.zeros(nb)
    cumulative[1:]=scipy.integrate.cumtrapz(1./e_z(zvals,cosmo),zvals)
    ### interpolation to input z values
    propdist=np.interp(z,zvals,cumulative)
    ### curvature
    omega=cosmo['omega_M_0']+cosmo['omega_lambda_0']
    k=np.abs(1-omega)
    if omega == 1:
        propdist=propdist
    elif omega < 1:
        propdist=np.sinh(np.sqrt(k)*propdist)/np.sqrt(k)
    elif omega > 1:
        propdist=np.sin(np.sqrt(k)*propdist)/np.sqrt(k)
    ### returning
    return(propdist*2.99792458e5/100/cosmo['h'])


In [15]:
cosmo_lcdm = {'h':0.7, 'omega_M_0':0.3, 'omega_lambda_0':0.7, 'w0':-1.}
cosmo_flat = {'h':0.7, 'omega_M_0':1, 'omega_lambda_0':0, 'w0':-1.}
cosmo_open = {'h':0.7, 'omega_M_0':0.3, 'omega_lambda_0':0, 'w0':-1.}

errorbar(zs, mm, yerr=dm, fmt='ko')
zvalues = linspace(0,2,10000)
plot(zvalues,musn1a(zvalues, cosmo_flat), lw=2, color='blue', 
     label='Flat matter only')
plot(zvalues,musn1a(zvalues, cosmo_open), lw=2, color='green', 
     label='Open matter only')
plot(zvalues,musn1a(zvalues, cosmo_lcdm), lw=2, color='red', 
     label='$\Lambda$CDM')
xscale('log')
xlim(1e-2,2)
ylim(33,46)
xlabel('Redshift')
ylabel('Distance Modulus')
legend(loc='upper left')


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: object of type <type 'numpy.float64'> cannot be safely interpreted as an integer.
  


TypeError: 'numpy.float64' object cannot be interpreted as an index

Error in callback <function post_execute at 0x7fbdb5ad0050> (for post_execute):


OSError: [Errno 2] No such file or directory: 'latex'

OSError: [Errno 2] No such file or directory: 'latex'

<Figure size 432x288 with 1 Axes>